In [1]:
# Initial imports
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from joblib import dump,load

In [2]:
Combined_Data = pd.read_csv("Resources/Cleaned CSVs/Combined_Data.csv")
df = pd.read_csv("Resources/Cleaned CSVs/Combined_Data.csv")

df_2024 = pd.read_csv("Resources/Cleaned CSVs/Cleaned_2024.csv")

Combined_Data.drop('Tm', axis=1, inplace=True)

# df_2024.drop("Unnamed: 0", axis=1, inplace=True)
# df_2024 = df_2024.drop([30, 31])

In [3]:
# data2scale = Combined_Data.drop('playoffs', axis=1)

In [4]:
# # Create a StandardScaler instance
# scaler = StandardScaler()

# # Scale all columns in the Combined_Data DataFrame
# Combined_scaled = scaler.fit_transform(data2scale)
# scaled_df = pd.DataFrame(Combined_scaled, columns=data2scale.columns)

# # Display the scaled DataFrame
# scaled_df

In [5]:
# scaled_df['playoffs'] = Combined_Data['playoffs']

# scaled_df

In [6]:
corr_data = Combined_Data.corr()['playoffs']

low_corr_df = pd.DataFrame(corr_data.loc[(corr_data <= 0.1) & (corr_data >= -0.1)])
low_corr_df.rename(columns={'playoffs': 'correlation'}, inplace=True)

index_values = low_corr_df.index.tolist()

index_values

['BK', 'CG_y', 'Ch', 'GF', 'SB', 'cSho']

In [7]:
columns_to_keep = index_values + ['playoffs']

focused_df = Combined_Data[columns_to_keep]
# scaled_df_focused = scaled_df[columns_to_keep]

focused_df

,BK,CG_y,Ch,GF,SB,cSho,playoffs
0,5.0,2.0,6150.0,160.0,79.0,1.0,0
1,8.0,2.0,5889.0,160.0,90.0,1.0,1
2,8.0,2.0,5934.0,160.0,81.0,0.0,0
3,3.0,2.0,5819.0,160.0,125.0,0.0,1
4,3.0,1.0,6208.0,162.0,66.0,0.0,1
...,...,...,...,...,...,...,...
145,7.0,0.0,5944.0,162.0,101.0,0.0,0
146,1.0,0.0,5802.0,162.0,160.0,0.0,1
147,4.0,3.0,5789.0,159.0,79.0,1.0,1
148,12.0,1.0,5733.0,161.0,99.0,1.0,1


 ## Separate the Features (X) from the Target (y)

In [8]:
y = focused_df["playoffs"]
X = focused_df.drop(columns="playoffs")

 ## Split our data into training and testing

In [9]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

X_train.shape

(112, 6)

In [10]:
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

 ## Create a Logistic Regression Model

In [11]:
classifier = LogisticRegression(solver='liblinear', max_iter=82)

classifier

LogisticRegression(max_iter=82, solver='liblinear')

 ## Fit (train) or model using the training data

In [12]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=82, solver='liblinear')

 ## Score the model using the test data

In [13]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.6696428571428571
Testing Data Score: 0.6578947368421053


 ## Make predictions

# Save the pre-trained model

In [14]:
dump(classifier, 'log_model.joblib')

['log_model.joblib']

In [15]:
model = load('log_model.joblib')

In [16]:
predictions_2024 = model.predict(df_2024[index_values])
predictions_2024

/Users/Jake/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
df_2024['predictions'] = predictions_2024
perdicted_2024 = df_2024[["Tm", "predictions"]]

perdicted_2024

,Tm,predictions
0,Arizona Diamondbacks,0
1,Atlanta Braves,0
2,Baltimore Orioles,0
3,Boston Red Sox,0
4,Chicago Cubs,0
5,Chicago White Sox,0
6,Cincinnati Reds,0
7,Cleveland Guardians,0
8,Colorado Rockies,0
9,Detroit Tigers,0


In [18]:
# Make list for all the predictions
predictions = {}

# Create loop for it to run 100 times with the model
iterations = 100
for i in range(iterations):
    y = focused_df["playoffs"]
    X = focused_df.drop(columns="playoffs")
    X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)
    classifier = LogisticRegression(solver='liblinear', max_iter=93)
    classifier.fit(X_train, y_train)
#     print(f"Training Data Score: {classifier.score(X_train, y_train)}")
#     print(f"Testing Data Score: {classifier.score(X_test, y_test)}")
    dump(classifier, 'log_model.joblib')
    model = load('log_model.joblib')
    predictions_2024 = model.predict(df_2024[index_values])
    predictions_df = pd.DataFrame({"Team": df_2024["Tm"], "Prediction": predictions_2024})
    counts = predictions_df.groupby('Team')['Prediction'].sum().astype(int)
    
    for team, count in counts.items():
        if team in predictions:
            predictions[team] += count
        else:
            predictions[team] = count

predictions

{'Arizona Diamondbacks': 0,
 'Atlanta Braves': 3,
 'Baltimore Orioles': 0,
 'Boston Red Sox': 25,
 'Chicago Cubs': 3,
 'Chicago White Sox': 13,
 'Cincinnati Reds': 69,
 'Cleveland Guardians': 6,
 'Colorado Rockies': 3,
 'Detroit Tigers': 0,
 'Houston Astros': 9,
 'Kansas City Royals': 14,
 'Los Angeles Angels': 8,
 'Los Angeles Dodgers': 0,
 'Miami Marlins': 1,
 'Milwaukee Brewers': 33,
 'Minnesota Twins': 10,
 'New York Mets': 0,
 'New York Yankees': 6,
 'Oakland Athletics': 0,
 'Philadelphia Phillies': 29,
 'Pittsburgh Pirates': 0,
 'San Diego Padres': 10,
 'San Francisco Giants': 0,
 'Seattle Mariners': 1,
 'St. Louis Cardinals': 0,
 'Tampa Bay Rays': 33,
 'Texas Rangers': 0,
 'Toronto Blue Jays': 6,
 'Washington Nationals': 43}

In [19]:
# Make list for all the predictions
scores = {}

# Create loop for it to run 100 times with the model
iterations = 100
for i in range(iterations):
    y = focused_df["playoffs"]
    X = focused_df.drop(columns="playoffs")
    X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)
    classifier = LogisticRegression(solver='liblinear', max_iter=93)
    classifier.fit(X_train, y_train)
    print(f"Training Data Score: {classifier.score(X_train, y_train)}")
    print(f"Testing Data Score: {classifier.score(X_test, y_test)}")
    dump(classifier, 'log_model.joblib')
    model = load('log_model.joblib')
    predictions_2024 = model.predict(df_2024[index_values])
    predictions_df = pd.DataFrame({"Team": df_2024["Tm"], "Prediction": predictions_2024})
    counts = predictions_df.groupby('Team')['Prediction'].sum().astype(int)
    
    for score, count in counts.items():
        if team in predictions:
            predictions[team] += count
        else:
            predictions[team] = count

predictions

Training Data Score: 0.6339285714285714
Testing Data Score: 0.631578947368421
Training Data Score: 0.6428571428571429
Testing Data Score: 0.6578947368421053
Training Data Score: 0.6071428571428571
Testing Data Score: 0.5526315789473685
Training Data Score: 0.6339285714285714
Testing Data Score: 0.6578947368421053
Training Data Score: 0.6428571428571429
Testing Data Score: 0.6578947368421053
Training Data Score: 0.6428571428571429
Testing Data Score: 0.6578947368421053
Training Data Score: 0.6428571428571429
Testing Data Score: 0.6578947368421053
Training Data Score: 0.6428571428571429
Testing Data Score: 0.631578947368421
Training Data Score: 0.6428571428571429
Testing Data Score: 0.6052631578947368
Training Data Score: 0.6428571428571429
Testing Data Score: 0.6578947368421053
Training Data Score: 0.6517857142857143
Testing Data Score: 0.5526315789473685
Training Data Score: 0.6428571428571429
Testing Data Score: 0.631578947368421
Training Data Score: 0.6607142857142857
Testing Data Sc

{'Arizona Diamondbacks': 0,
 'Atlanta Braves': 3,
 'Baltimore Orioles': 0,
 'Boston Red Sox': 25,
 'Chicago Cubs': 3,
 'Chicago White Sox': 13,
 'Cincinnati Reds': 69,
 'Cleveland Guardians': 6,
 'Colorado Rockies': 3,
 'Detroit Tigers': 0,
 'Houston Astros': 9,
 'Kansas City Royals': 14,
 'Los Angeles Angels': 8,
 'Los Angeles Dodgers': 0,
 'Miami Marlins': 1,
 'Milwaukee Brewers': 33,
 'Minnesota Twins': 10,
 'New York Mets': 0,
 'New York Yankees': 6,
 'Oakland Athletics': 0,
 'Philadelphia Phillies': 29,
 'Pittsburgh Pirates': 0,
 'San Diego Padres': 10,
 'San Francisco Giants': 0,
 'Seattle Mariners': 1,
 'St. Louis Cardinals': 0,
 'Tampa Bay Rays': 33,
 'Texas Rangers': 0,
 'Toronto Blue Jays': 6,
 'Washington Nationals': 392}